### Imports

In [441]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from arch import arch_model
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor

plt.rcParams["figure.figsize"] = (12, 7)

### Getting the data

In [4]:
def get_sp500_ticker_list():
    """
    Returns a list with all SP500 tickers
    """
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    table = tables[0]
    ticker_list = table['Symbol']
    return ticker_list

In [470]:
def get_sample_ticker(ticker_list, s=5):
    sample = ticker_list.sample(s).to_list()
    return sample

In [11]:
def get_adj_close(ticker_list, start, end, interval):
    """
    Returns the adjusted close for a unique ticker as string or a list of tickers.
    Format of dates: 'yyyy-mm-dd'
    Possible intervals: '1d', '5d', '1mo' 
    or intraday measures but limited to max a week's worth: '1m', '2m', '5m', '15m', '30m'
    """
    full_df = yf.download(ticker_list, start=start, end=end, interval=interval)
    adj_close_df = full_df['Adj Close']
    return adj_close_df

In [471]:
def get_adj_close_df(s=5, start='2020-01-01', end='2021-12-31', interval='1d'):
    """
    Returns the returns df and adj close df for a unique ticker as string or a list of tickers.
    n = sample size
    Format of dates: 'yyyy-mm-dd'
    Possible intervals: '1d', '5d', '1mo' 
    or intraday measures but limited to max a week's worth: '1m', '2m', '5m', '15m', '30m'
    Returns a dataframe of a random sample of the sp500 adj closes over a certain period of time
    """
    sp500_tickers = get_sp500_ticker_list()
    sample_tickers = get_sample_ticker(sp500_tickers, s)
    full_df = yf.download(sample_tickers, start=start, end=end, interval=interval)
    adj_close_df = full_df['Adj Close']
    return adj_close_df

In [64]:
adj_close_df = get_adj_close_df()

[*********************100%***********************]  5 of 5 completed


In [63]:
def get_returns(adj_close_df):
    df_returns = (adj_close_df.pct_change())*100
    df_returns.dropna(axis=0,inplace=True)
    return df_returns

In [65]:
returns_df = get_returns(adj_close_df)

In [67]:
def get_volatility(returns_df):
    realized_vol = returns_df.rolling(5).std()
    realized_vol.dropna(inplace=True)
    return realized_vol

In [342]:
volatility_df = get_volatility(returns_df)

### Splitting the dataset by observations

In [158]:
def split_df(df, n=50):
    df_test = df.iloc[-n:]
    df_train = df.iloc[:-n]
    split_date = df.iloc[-n:].index
    return df_train, df_test, split_date

In [159]:
df_train_vol, df_test_vol, split_date = split_df(volatility_df)
df_train_ret, df_test_ret, split_date = split_df(returns_df)

### GARCH

In [280]:
def garch(returns, n):

    aic_garch = []

    for p in range(1, 2): 
        for q in range(1, 2):
            garch = arch_model(returns, mean='zero', vol='GARCH', p=p, q=q)\
                .fit(disp='off') 
            aic_garch.append(garch.aic) 

            if garch.aic == np.min(aic_garch): 
                best_param = (p,q) 
    
    #fitting the best GARCH model
    garch = arch_model(returns, mean='zero', vol='GARCH', p=best_param[0], q=best_param[1]).fit(disp='off')

    #forecasts
    forecasts = garch.forecast(horizon=n, reindex=False)
    #forecasts = garch.forecast(horizon=50, start=split_date[0], reindex=True)
    return forecasts, forecasts.residual_variance.dropna().transpose()

In [281]:
forecasts, residuals = garch(df_train_ret.iloc[:, :1], 50)

In [485]:
def get_rmse(residuals, realized_vol):
    rmse = np.sqrt(mse(realized_vol/100, np.sqrt(residuals/100)))
    return rmse

In [284]:
get_rmse(residuals, df_test)

0.2205656102776126

### ARCH

In [292]:
def arch(returns, n):
    aic_arch = []

    for p in range(1, 2): # Iterating ARCH parameter p
        arch = arch_model(returns, mean='zero', vol='ARCH', p=p)\
             .fit(disp='off') # Running ARCH(p)
        aic_arch.append(arch.aic) # Storing aic for the ARCH(p)

        if arch.aic == np.min(aic_arch): 
             best_param = p # Finding the minimum AIC score
                
    # Fitting best arch
    arch = arch_model(returns, mean='zero', vol='ARCH', p=best_param)\
         .fit(disp='off')
    
    forecasts = arch.forecast(horizon=n, reindex=False)
    
    return forecasts, forecasts.residual_variance.dropna().transpose()

In [293]:
forecasts, residuals = garch(df_train_ret.iloc[:, :1], 50)

In [294]:
get_rmse(residuals, df_test)

0.2205656102776126

### GJR garch

In [295]:
def gjr_garch(returns, n):
    aic_gjr_garch = []

    for p in range(1, 2): 
        for q in range(1, 2):
            gjr_garch = arch_model(returns, mean='zero', vol='GARCH', p=p, o=1, q=q)\
                 .fit(disp='off') 
            aic_gjr_garch.append(gjr_garch.aic) 

            if gjr_garch.aic == np.min(aic_gjr_garch): 
                 best_param = p, q # Finding the minimum AIC score
    
    gjr_garch = arch_model(returns, mean='zero', vol='ARCH', p=best_param[0], o=1,
                       q=best_param[1]).fit(disp='off')
    
    forecasts = gjr_garch.forecast(horizon=n, reindex=True)
    
    return forecasts, forecasts.residual_variance.dropna().transpose()

In [304]:
forecasts, residuals = gjr_garch(df_train_ret.iloc[:, :1], 50)

In [297]:
get_rmse(residuals, df_test)

0.20397548607626004

### EGARCH

In [326]:
def egarch(returns, n):
    aic_egarch = []

    for p in range(1, 2):
        for q in range(1, 2):
            egarch = arch_model(returns, mean='zero', vol='EGARCH', p=p, q=q)\
                  .fit(disp='off')
            aic_egarch.append(egarch.aic)
            if egarch.aic == np.min(aic_egarch):
                best_param = (p, q)
    
    egarch = arch_model(returns, mean='zero', vol='EGARCH',
                        p=best_param[0], q=best_param[1], dist="skewt").fit(disp='off')
    
    forecasts = egarch.forecast(horizon=50, method='simulation', reindex=False)
    
    return forecasts, forecasts.residual_variance.dropna().transpose()

In [327]:
forecasts, residuals = egarch(df_train_ret.iloc[:, :1], 50)

In [328]:
get_rmse(residuals, df_test)

0.23406873545174414

### Neural nets, preprocessing

In [351]:
def get_svm_volatility(volatility_df):
    realized_vol = {}
    for ticker in volatility_df.columns:
        realized_vol[ticker] = pd.DataFrame(volatility_df[ticker]).reset_index(drop=True)
    return realized_vol

In [372]:
def get_svm_returns(returns_df):
    returns_svm = {}
    for ticker in returns_df.columns:
        returns_svm[ticker] = returns_df[ticker]**2
        returns_svm[ticker] = returns_svm[ticker].reset_index()
        del returns_svm[ticker]['Date']
    return returns_svm

In [418]:
def concat_ret_vol(returns_df, volatility_df):
    Xs = {}
    realized_vol = get_svm_volatility(volatility_df)#[:-4]
    returns_svm = get_svm_returns(returns_df)
    for ticker in volatility_df.keys():
        Xs[ticker] = pd.concat([realized_vol[ticker], returns_svm[ticker][:-4]], axis=1, ignore_index=True)
        
    return Xs

In [426]:
Xs = concat_ret_vol(returns_df, volatility_df)

In [430]:
# def get_train_test_svm(Xs, n=50):
#     Xs_splitted = {}
#     for ticker, X in Xs.items():
#         Xs_splitted[ticker] = {}
#         Xs_splitted[ticker]['train'] = X.iloc[:-n]
#         Xs_splitted[ticker]['test'] = X.iloc[-n:]
#     return Xs_splitted

<!-- Xs_splitted = get_train_test_svm(Xs, n=50) -->

### Neural Nets

In [438]:
NN_vol = MLPRegressor(learning_rate_init=0.001, random_state=1)

In [439]:
para_grid_NN = {'hidden_layer_sizes': [(100, 50), (50, 50), (10, 100)],
                'max_iter': [500, 1000],
                'alpha': [0.00005, 0.0005 ]}

In [442]:
clf = RandomizedSearchCV(NN_vol, para_grid_NN)

In [451]:
def neural_net(Xs, n=50):
    X_predictions = {}
    for ticker, X in Xs.items():
        realized_vol = X[0]
        clf = RandomizedSearchCV(NN_vol, para_grid_NN)
        clf.fit(X.iloc[:-n].values,
            realized_vol.iloc[1:-(n-1)].values.reshape(-1, ))
        X_predictions[ticker] = clf.predict(X.iloc[-n:])
    return X_predictions

In [449]:
X_predictions = neural_net(Xs_splitted, n=50)

In [454]:
X_predictions['ADBE']

array([1.16530353, 1.22485074, 1.02747157, 0.90250107, 0.89371821,
       0.79185244, 0.73967724, 1.04380431, 1.25407036, 1.25634262,
       1.63844025, 1.68082139, 2.04213559, 1.93606472, 1.6898283 ,
       2.10746934, 1.32340398, 2.04412487, 2.0340602 , 1.87257907,
       1.26845727, 0.95259769, 1.24938606, 1.68516381, 1.8859812 ,
       1.9106961 , 1.67152201, 2.3004118 , 2.52611034, 2.67652743,
       2.90530969, 4.7545577 , 4.21062848, 5.16119125, 5.13596104,
       3.65973335, 3.1468498 , 2.55450584, 3.98834018, 4.34798576,
       6.48210942, 5.2853198 , 4.25302711, 5.26657878, 3.12514948,
       1.30989628, 1.05252282, 1.07423454, 1.13466524, 1.10235344])

In [459]:
rmse = np.sqrt(mse(df_test['ADBE'] / 100,
                   X_predictions['ADBE'] / 100))

In [460]:
rmse

0.0033607729900200665

In [492]:
def get_rmses(s, n, start, end, interval):
    """
    s = how many companies
    n = sample size for testing
    """
    NN_vol = MLPRegressor(learning_rate_init=0.001, random_state=1)
    para_grid_NN = {'hidden_layer_sizes': [(100, 50), (50, 50), (10, 100)],
                'max_iter': [500, 1000],
                'alpha': [0.00005, 0.0005 ]}
    rmses = {}
    
    # Get data
    adj_close_df = get_adj_close_df(s=s, start=start, end=end, interval=interval)
    
    # Get returns and volatility for arch type models
    returns_df = get_returns(adj_close_df)
    volatility_df = get_volatility(returns_df)
    
    # Splitting sets for arch type models
    df_train_vol, df_test_vol, split_date = split_df(volatility_df, n)
    df_train_ret, df_test_ret, split_date = split_df(returns_df, n)
    
    # Neural net preprocessing
    Xs = concat_ret_vol(returns_df, volatility_df)
    nn_predictions = neural_net(Xs, n=n)
    
    # Getting rmses
    for ticker in df_test_vol.columns:
        rmses[ticker] = {}
        
        #Neural net
        rmses[ticker]['nn'] = np.sqrt(mse(df_test_vol[ticker] / 100, nn_predictions[ticker] / 100))
        
#         #Garch
#         forecasts, residuals = garch(df_train_ret[ticker], n)
#         rmses[ticker]['garch'] = get_rmse(residuals, df_test_vol[ticker])
        
#         #Arch
#         forecasts, residuals = garch(df_train_ret[ticker], n)
#         rmses[ticker]['arch'] = get_rmse(residuals, df_test_vol[ticker])
        
#         #GJR-garch
#         forecasts, residuals = gjr_garch(df_train_ret[ticker], n)
#         rmses[ticker]['gjr_garch'] = get_rmse(residuals, df_test_vol[ticker])
        
#         #Egarch
#         forecasts, residuals = egarch(df_train_ret[ticker], n)
#         rmses[ticker]['egarch'] = get_rmse(residuals, df_test_vol[ticker])
        
    return rmses

In [ ]:
get_rmses(s=2, n=50, start='2020-01-01', end='2021-12-31', interval='1d')

[*********************100%***********************]  2 of 2 completed
